In [1]:
from qutip import *
import numpy as np
import scipy.optimize as sciop
import matplotlib.pyplot as plt
import cython
import skopt
import multiprocessing
from joblib import Parallel, delayed
from tqdm import tqdm
import pickle

# define idealised basis
basis_labels = {
    'cav_bin': 0,
    'spont_bin': 1,
    'g':2,
    'u':3,
    'x':4
}
N=len(basis_labels)

g_ket = basis(N, basis_labels['g'])
u_ket = basis(N, basis_labels['u'])
x_ket = basis(N, basis_labels['x'])

# define basic operators
gx_swap = basis(N, basis_labels['g']) * basis(N, basis_labels['x']).dag()
ux_swap = basis(N, basis_labels['u']) * basis(N, basis_labels['x']).dag()

g_id = fock_dm(N, basis_labels['g'])
u_id = fock_dm(N, basis_labels['u'])
x_id = fock_dm(N, basis_labels['x'])
cav_id = fock_dm(N, basis_labels['cav_bin'])
spont_id = fock_dm(N, basis_labels['spont_bin'])

cav_decay = basis(N, basis_labels['cav_bin']) * basis(N, basis_labels['g']).dag()
spont_decay = basis(N, basis_labels['spont_bin']) * basis(N, basis_labels['x']).dag()

In [2]:
# define pulse shape
H_args = {}

sin_pulse = 'omega*np.sin(wSTIRAP*t)**2'

opts = Options(rhs_reuse=True)

In [3]:
# define parameters
gamma = 2*np.pi * 1
kappa = gamma * 1

# define collapse operators
c_ops = [np.sqrt(gamma)*spont_decay, np.sqrt(kappa)*cav_decay]

STEPS_PER_SEC = 1000


In [4]:
# define coordinates
NUM_COOPS = 25
g_min = np.sqrt(2*kappa/gamma)
g_max = 10*np.sqrt(4*kappa/gamma)

g_ratios = np.logspace(g_min, g_max, NUM_COOPS)

NUM_TIMES = 25
t_min = 0.1
t_max = 100

times = np.logspace(t_min, t_max, NUM_TIMES)

omega_guesses = [1000]*len(g_ratios)
max_deltas = [200]*len(g_ratios)

current_time = t_min

In [7]:
omega_range_factor = 5
num_bayesian_calls = 50

def find_efficiency(g_ratio, omega, delta, time, psi0=u_ket, pulse_shape=sin_pulse):

    omega *= 2*np.pi
    delta *= 2*np.pi
    H_args['omega'] = omega
    g0 = g_ratio * gamma

    global current_time

    if time != current_time:
        opts.rhs_reuse = False
        current_time = time

    # define time steps
    runtime = time/gamma
    H_args['T'] = runtime
    H_args['wSTIRAP'] = np.pi/runtime
    num_steps = round(STEPS_PER_SEC*runtime + 1)

    t_res = 2
    int_time = round(runtime, t_res)*10**t_res
    t = np.linspace(0, int_time, num_steps)
    t /= 10**t_res

    H0 = delta*g_id + delta*u_id - g0*(gx_swap + gx_swap.dag())
    H1 = -1/2*(ux_swap + ux_swap.dag())
    H=[H0, [H1, pulse_shape]]

    result = mesolve(H, psi0, t, c_ops, [], args=H_args, options=opts)

    photon_emission_prob = expect(cav_id, result.states[-1])
    
    opts.rhs_reuse = True

    return photon_emission_prob


def bayesian_optimiser(g_ratio, omega_guess, delta_max, time):

    def find_neg_efficiency(omega_delta, psi0=u_ket, pulse_shape=sin_pulse):

        omega = 2*np.pi * omega_delta[0]
        H_args['omega'] = omega
        delta = 2*np.pi * omega_delta[1]
        g0 = g_ratio * gamma

        global current_time

        if time != current_time:
            opts.rhs_reuse = False
            current_time = time

        # define time steps
        runtime = time/gamma
        H_args['T'] = runtime
        H_args['wSTIRAP'] = np.pi/runtime
        num_steps = round(STEPS_PER_SEC*runtime + 1)

        t_res = 2
        int_time = round(runtime, t_res)*10**t_res
        t = np.linspace(0, int_time, num_steps)
        t /= 10**t_res

        H0 = delta*g_id + delta*u_id - g0*(gx_swap + gx_swap.dag())
        H1 = -1/2*(ux_swap + ux_swap.dag())
        H=[H0, [H1, pulse_shape]]

        result = mesolve(H, psi0, t, c_ops, [], args=H_args, options=opts)

        opts.rhs_reuse = True
        return -expect(cav_id, result.states[-1])
    
    return skopt.gp_minimize(find_neg_efficiency, [[0, omega_guess*omega_range_factor], [0, delta_max]], x0=[omega_guess, 0], n_calls=num_bayesian_calls)


def iterative_optimiser(g_ratio, omega_guess, delta, time):

    resolution_level = [5,1,0.5,0.1]

    initial_eff = find_efficiency(g_ratio, omega_guess, delta, time)

    for res in resolution_level:
        print('new')
        finished = False
        new_step = 0

        while not finished:

            if new_step == 0:
                temp_effs = [initial_eff, find_efficiency(g_ratio, omega_guess-res, delta, time), find_efficiency(g_ratio, omega_guess + res, delta, time)]
            elif new_step == 1:
                temp_effs = [initial_eff, temp_effs[0], find_efficiency(g_ratio, omega_guess+res, delta, time)]
            else:
                temp_effs = [initial_eff, find_efficiency(g_ratio, omega_guess-res, delta, time), temp_effs[0]]
            

            if np.argmax(temp_effs) == 0:
                finished = True
            elif np.argmax(temp_effs) == 1:
                new_step = -1
            else:
                new_step = 1

    if omega_guess < 0:
        omega_0 *= -1
    
    return[omega_guess, initial_eff]


def combined_optimiser(inputs):
    g_ratio, omega_guess, delta_max, time = inputs

    bayesian_op_data = bayesian_optimiser(g_ratio, omega_guess, delta_max, time)
    bayes_omg, delta = bayesian_op_data.x
    print('yes')
    opt_omg, opt_eff = iterative_optimiser(g_ratio, omega_guess, delta, time)

    return {
        'g_ratio': g_ratio,
        'time': time,
        'omega': opt_omg,
        'delta': delta,
        'efficiency': opt_eff
    }



In [8]:
def save_obj(object, name):
    with open(name+'.pickle', 'wb') as f:
        print(name+'.pickle')
        pickle.dump(object, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_obj(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

In [ ]:
max_delta_factor = 8

# optimisation
total_cores = multiprocessing.cpu_count()
if total_cores > 60:
    num_cores = 60
else:
    num_cores = total_cores

input_args = []

for j in range(len(times)):

    sub_input_args = [[g_ratios[k], omega_guesses[k], max_deltas[k], times[j]] for k in range(len(g_ratios))]
    input_args.append(sub_input_args)

output_list = []
first_run = True
for input_set in input_args:
    
    if not first_run:
        for k in range(len(input_set)):
            input_set[k][1] = processed_list[k]['omega']
            input_set[k][2] = processed_list[k]['delta']*max_delta_factor

    if __name__ == "__main__":
        inputs = tqdm(input_set)
        processed_list = Parallel(n_jobs=num_cores)(delayed(combined_optimiser)(i) for i in inputs)
        output_list.append(processed_list)
    first_run = False

 16%|█▌        | 4/25 [00:20<00:00, 28.95it/s]

In [ ]:
all_outputs = []
for data_set in output_list:
    sub_omgs = []
    sub_delts = []
    sub_effs = []
    for data in data_set:
        sub_omgs.append(data['omega'])
        sub_delts.append(data['delta'])
        sub_effs.append(data['efficiency'])
    all_outputs.append({
        'omegas': sub_omgs,
        'deltas': sub_delts,
        'efficiencies': sub_effs
    })

final_data = [g_ratios, times, all_outputs]
save_obj(final_data, 'time-varying sin^2 2D, kappa=1gamma, gamma=2pi')

In [ ]:
X, Y = np.meshgrid(g_ratios, times)
num_contours = 20

grand_efficiencies = [all_outputs[k]['efficiencies'] for k in range(len(all_outputs))]

fig, axes = plt.subplots(1,1)
cp = axes.contourf(X,Y, grand_efficiencies, levels = num_contours)

axes.set_xscale('log')
axes.set_yscale('log')

cbar = fig.colorbar(cp)
cbar.set_label('Efficiency')

bench_effs = []
for ratio in g_ratios:
    C = ratio**2*gamma/(2*kappa)
    bench_effs.append(2*C/(2*C+1))

grand_bench_effs = [bench_effs for j in range(len(times))]

grand_diffs = []
for j in range(len(times)):
    diff_effs = [grand_efficiencies[j][k] - grand_bench_effs[j][k] for k in range(len(g_ratios))]
    grand_diffs.append(diff_effs)

cont = axes.contour(X,Y,grand_diffs,[0], colors = 'k')
axes.set_xlabel('$g/\gamma$')
axes.set_ylabel('$\kappa T$')
axes.set_title('$\gamma=2\pi$, $\sin^2$ pulse, $kappa=\gamma$')

In [ ]:
cp = axes.contourf(X,Y, grand_diffs, levels = num_contours)

axes.set_xscale('log')
axes.set_yscale('log')

cbar = fig.colorbar(cp)
cbar.set_label('Efficiency Difference with Benchmark')
cont = axes.contour(X,Y,grand_diffs,[0], colors = 'k')
axes.set_xlabel('$g/\gamma$')
axes.set_ylabel('$\kappa T$')
axes.set_title('$\gamma=2\pi$, $\sin^2$ pulse, $kappa=\gamma$')